In [1]:
# Task description
""" 
from Crypto.Util.Padding import pad

BLOCK_SIZE = 8
key = b'????????'
f = open("secret.html", "rb")
plaintext = pad(f.read(),BLOCK_SIZE)
message = b''

key_it = 0
for c in plaintext:
    message += bytes([c ^ key[key_it]])
    key_it = (key_it + 1) % BLOCK_SIZE

import base64
print(base64.b64encode(message))
#"""

# What we know
# 
# The cipher begins with <html> and ends with </html>
# we need to take the first and last 4 characters to get the 8 long key
# 
# <hmtl>
# DmMhH3
# 
# </html>
# 9WXk1Yw==
#              01234567012345670123456701234567012345670123456701234567012345670123456701234567
#              <html>

ciphertext = b'DmMhH3YTjF3Yg0kWCsJaw4qET1ZZ1sAXCVYclApFV84fg01VCMJaw42DCRZIgkyVyEXNUEzC05GKxE8UHk9XR0qADFReT1dUC0BKQtNPV1CfDE4UGdUDlYnRTlGZxA1QgAVG1hxbRhgMSITXw1RRhx+SiALTT1dHSAKNEx5PV0dKhE9WXk1Yw=='

In [2]:
# Helper functions

def fillupbyte(s, padn = 8, padchar = '0', mode = 'l'):
     n = len(s)
     l = n
     while(l % padn != 0):
         l += 1
     result = ''
     j = 0
     for i in range(l):
         if(i < l-n):
             if(mode == 'l'):
                 result = padchar + result
             else:
                 result = result  + padchar
         else:
             if(mode == 'l'):
                 result = result + s[j]
                 j += 1
             else:
                 result = s[::-1][j] + result
                 j += 1
     return result

def convertbin(m):
    result = ''
    for i in range(len(m)):
        result += fillupbyte(bin(m[i])[2:], 8)
    return result

def arr_xor(arr1, arr2):
    target_len = max(len(arr1),len(arr2))
    result = ''
    for i in range(target_len):
        if arr1[i] == arr2[i]:
            result += "0"
        else:
            result += "1"
    return result

def bin2ascii(binarr):
    result = ''
    for i in range(0,len(binarr),8):
        chunk = binarr[i:i+8]
        chunk_int = int(chunk,2)
        result += chr(chunk_int)
    return result

In [3]:
# Getting the key

begin = b'<html>' # We assume this is the beginning of the key. The key is a 8-bit array 
                  # that gets repeated. Every 8th bit is xored with the same bit of the key
begin_bin = convertbin(begin) # Convert the beginning of the key to binary (to be able to xor)
begin_secret = ciphertext[:len(begin)] # Get the corresponding encrypted text from the ciphertext
begin_secret_bin = convertbin(begin_secret) # Convert the beginning of the cypher to binary

# THE ONLY QUESTION IS WHAT IS END, AND END_SECRET. IT'S A GUESSING GAME AT THIS POINT
_ = '</html>' 
end = b'</' # We assume this is the end of our key. As the beginning <html> 
            # is 6 characters long, and the block size is 8, we only need 2 characters. 
minus_ind = 0 # This is just a shift variable corresponding to how much padding was there on the message before encryption
end_bin = convertbin(end) # Convert the secret key to binary
end_secret = ciphertext[-len(end) - minus_ind : ] # Get the corresponging encrypted message
end_secret_bin = convertbin(end_secret) # Convert corresponding encrypted secret to binary

begin_key_bin = arr_xor(begin_bin, begin_secret_bin) # xor the beginning of the message with the beginning of the key
end_key_bin = arr_xor(end_bin, end_secret_bin) # xor the beginning of the message with the end of the key

key_bin = begin_key_bin + end_key_bin # To get the key we concatenate the beginning and the end

key = b'' # Create an empty byte array for the key
for i in range(0, len(key_bin), 8): # Iterate over the binary message in 8-blocks. 
                                    # 8bit=1byte corresponds to 1 single character in the encrypted message
    chunk = key_bin[i:i+8] # Get the next 8 bytes to decrypt
    key += bytes([int(chunk,2)]) # Convert to byte to integer integer

In [4]:
# Convert original message
BLOCK_SIZE = 8
result = b''
key_it = 0

for c in ciphertext: # As we have the key now we can iterate over the encrypted message in 1-blocks
    result += bytes([c ^ key[key_it]]) # Xor the encrypted message with the corresponding key part
    key_it = (key_it + 1) % BLOCK_SIZE # Get the next key to xor with. 

print(result)

b'<html>XF\x12C\n\\C=jE;vsdS9pW,4c_\x15~@J;S`fHLqT.=\rcC=0D;HsdS93V;WcLCfxD\x01@aKqH{QH0~N\\H9G0n\x00]v=pS<Ck`p<eG;5{NuyOB.4zc`H5G?alAHTo@,i~_\\L0C\x1fDoB\x15eyp*m^HwDUJ\x0f4kWLu*A\x11DuQp<eZ+DrKau4B.5]NLH8E n\x08\\S0<'
